<a href="https://colab.research.google.com/github/ptmhoang97/fiftyone_coco/blob/main/main_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
COCO_DATASET_YEAR = '2017'
COCO_DATASET_TYPE = 'train'
COCO_DATASET_CLASS = 'car'

#1. Install FiftyOne

In [ ]:
import fiftyone

In [ ]:
!fiftyone --version

In [ ]:
# The version that you wish to downgrade to
#!fiftyone migrate --all -v 0.11.1
#!fiftyone migrate --info

In [ ]:
!pip install fiftyone

#2. Download images and generate annotation files

In [ ]:
import fiftyone.zoo as foz

In [ ]:
import os
print(os.name)

In [ ]:
result_name = 'coco{}_{}_{}'.format(COCO_DATASET_YEAR,COCO_DATASET_TYPE,COCO_DATASET_CLASS)
result_name_zip = '{}.zip'.format(result_name)
print(result_name)
print(result_name_zip)

In [ ]:
paths = {
    'COCO_DATASET_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR)),
    'RESULT_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR), result_name),
 }

In [ ]:
dataset = foz.load_zoo_dataset(name = 'coco-{}'.format(COCO_DATASET_YEAR),\
                               classes = COCO_DATASET_CLASS,\
                               split = COCO_DATASET_TYPE,\
                               dataset_dir = paths['COCO_DATASET_PATH'])

In [ ]:
!mkdir {paths['RESULT_PATH']}

In [ ]:
from pycocotools.coco import COCO
import os
import shutil
from tqdm import tqdm
# import skimage.io as io
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw

In [ ]:
resultDir=paths['RESULT_PATH']
srcImageDir = paths['COCO_DATASET_PATH'] + '/{}/{}'.format(COCO_DATASET_TYPE,'data')
srcAnnoDir = paths['COCO_DATASET_PATH'] + '/{}'.format('raw')
datasets_list=[]
if COCO_DATASET_TYPE == 'validation':
    temp = 'val' + COCO_DATASET_YEAR
    datasets_list = [temp]
elif COCO_DATASET_TYPE == 'train':
    temp = 'train' + COCO_DATASET_YEAR
    datasets_list = [temp]
classes_names = [COCO_DATASET_CLASS]
print(datasets_list)
print(classes_names)
print("resultDir: " + str(resultDir))
print("srcImageDir: " + str(srcImageDir))
print("srcAnnoDir: " + str(srcAnnoDir))

In [ ]:
headstr = """\
<annotation>
    <folder>VOC</folder>
    <filename>%s</filename>
    <source>
        <database>My Database</database>
        <annotation>COCO</annotation>
        <image>flickr</image>
        <flickrid>NULL</flickrid>
    </source>
    <owner>
        <flickrid>NULL</flickrid>
        <name>company</name>
    </owner>
    <size>
        <width>%d</width>
        <height>%d</height>
        <depth>%d</depth>
    </size>
    <segmented>0</segmented>
"""
objstr = """\
    <object>
        <name>%s</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>%d</xmin>
            <ymin>%d</ymin>
            <xmax>%d</xmax>
            <ymax>%d</ymax>
        </bndbox>
    </object>
"""
 
tailstr = '''\
</annotation>
'''
 
#if the dir is not exists,make it,else delete it
def id2name(coco):
    classes=dict()
    for cls in coco.dataset['categories']:
        classes[cls['id']]=cls['name']
    return classes
 
def write_xml(anno_path,head, objs, tail):
    f = open(anno_path, "w")
    f.write(head)
    for obj in objs:
        f.write(objstr%(obj[0],obj[1],obj[2],obj[3],obj[4]))
    f.write(tail)
 
def save_annotations_and_imgs(coco,dataset,filename,objs):
    #eg:COCO_train2014_000000196610.jpg-->COCO_train2014_000000196610.xml
    result_anno_path = '{}/{}{}'.format(resultDir,filename[:-3],'xml')
    #print(result_anno_path)
    src_img_path = '{}/{}'.format(srcImageDir,filename)
    #print(anno_path)
    #print(src_img_path)
    dst_imgpath = '{}/{}'.format(resultDir,filename)
    #print(dst_imgpath)
    img=cv2.imread(src_img_path)
    #if (img.shape[2] == 1):
    #    print(filename + " not a RGB image")
     #   return
    shutil.copy(src_img_path, dst_imgpath)
 
    head=headstr % (filename, img.shape[1], img.shape[0], img.shape[2])
    tail = tailstr
    write_xml(result_anno_path,head, objs, tail)
 
 
def showimg(coco,dataset,img,classes,cls_id,show=True):
    #global dataDir
    path = '{}/{}'.format(srcImageDir,img['file_name'])
    #print(path)
    I=Image.open(path)
    #By id, get comment information
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=cls_id, iscrowd=None)
    # print(annIds)
    anns = coco.loadAnns(annIds)
    # print(anns)
    # coco.showAnns(anns)
    objs = []
    for ann in anns:
        class_name=classes[ann['category_id']]
        if class_name in classes_names:
            #print(class_name)
            if 'bbox' in ann:
                bbox=ann['bbox']
                xmin = int(bbox[0])
                ymin = int(bbox[1])
                xmax = int(bbox[2] + bbox[0])
                ymax = int(bbox[3] + bbox[1])
                obj = [class_name, xmin, ymin, xmax, ymax]
                objs.append(obj)
                draw = ImageDraw.Draw(I)
                draw.rectangle([xmin, ymin, xmax, ymax])
    if show:
        plt.figure()
        plt.axis('off')
        plt.imshow(I)
        plt.show()
 
    return objs

for dataset in datasets_list:
    #./COCO/annotations/instances_train2014.json
    annFile='{}/instances_{}.json'.format(srcAnnoDir,dataset)
    #print(annFile)
    #COCO API for initializing annotated data
    coco = COCO(annFile)
 
    #show all classes in coco
    classes = id2name(coco)
    #print(classes)
    #[1, 2, 3, 4, 6, 8]
    classes_ids = coco.getCatIds(catNms=classes_names)
    #print(classes_ids)
    for cls in classes_names:
        #Get ID number of this class
        cls_id=coco.getCatIds(catNms=[cls])
        img_ids=coco.getImgIds(catIds=cls_id)
        #print(cls,len(img_ids))
        # imgIds=img_ids[0:10]
        for imgId in tqdm(img_ids):
            img = coco.loadImgs(imgId)[0]
            filename = img['file_name']
            # print(filename)
            objs=showimg(coco, dataset, img, classes,classes_ids,show=False)
            #print(objs)
            save_annotations_and_imgs(coco, dataset, filename, objs)

#3. Zip and copy dataset to GG drive

In [ ]:
%cd {paths['COCO_DATASET_PATH']}
!ls

In [ ]:
!zip -r result_name_zip result_name

In [ ]:
!cp {result_name_zip} "/content/gdrive/MyDrive"